# Quickstart

This section runs through the API for common tasks in machine learning. Refer to the links in each section to dive deeper.  

### Working with data
PyTorch has two primitives to work with data: ``torch.utils.data.DataLoader`` and ``torch.utils.data.Dataset.``   
데이터 작업을 위한 두 가지 기본 요소 : torch.utils.data.DataLoader , torch.utils.data.Dataset  

`Dataset` stores the samples and their corresponding labels,   
샘플과 해당하는 레이블을 저장  

and `DataLoader` wraps an iterable around the Dataset.  
dataset을 순회하는 기능을 제공

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch offers domain-specific libraries such as [TorchText](https://pytorch.org/text/stable/index.html),
[TorchVision](https://pytorch.org/vision/stable/index.html), and [TorchAudio](https://pytorch.org/audio/stable/index.html), all of which include datasets.   
PyTorch는 TorchText , TorchVision 및 TorchAudio 와 같은 도메인별 라이브러리를 제공하며 모두 데이터 세트를 포함  

For this tutorial, we  will be using a TorchVision dataset.  

The ``torchvision.datasets`` module contains ``Dataset`` objects for many real-world vision data like CIFAR, COCO ([full list here](https://pytorch.org/vision/stable/datasets.html)).   
torchvision.datasets모듈에는 CIFAR, COCO와 같은 많은 실제 비전 데이터에 대한 개체가 포함되어 있음  

In this tutorial, we use the FashionMNIST dataset.   

Every TorchVision ``Dataset`` includes two arguments: ``transform`` and
``target_transform`` to modify the samples and labels respectively.  
모든 TorchVision에는 샘플과 라벨을 각각 수정하는 두 가지 인수가 포함되어 있음 : Dataset transform, target_transform


In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(), # 데이터를 텐서로 저장
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



We pass the ``Dataset`` as an argument to ``DataLoader``.   
Dataset을 DataLoader의 인자로 전달  

This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading.   
이는 데이터셋에 대한 반복 가능한(iterable) 객체를 래핑하며, 자동으로 배치 처리, 샘플링, 셔플링 및 다중 프로세스 데이터 로딩을 지원  

Here we define a batch size of 64, i.e. each element in the dataloader iterable will return a batch of 64 features and labels.
데이터로더(iterable)의 각 요소는 64개의 피처와 레이블로 구성된 배치를 반환

In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


---

### Creating Models
To define a neural network in PyTorch, we create a class that inherits from ``nn.Module``.  
PyTorch에서 신경망을 정의하려면 nn.Module에서 상속받은 클래스를 생성  

We define the layers of the network in the ``__init__`` function and specify how data will pass through the network in the forward function.   
함수에서 네트워크의 계층을 정의 __init__하고  
함수에서 데이터가 네트워크를 통과하는 방법을 지정 (forward)  

To accelerate operations in the neural network, we move it to the GPU or MPS if available. 
신경망의 연산을 가속화하기 위해 가능한 경우 GPU 또는 MPS로 이동  

MPS : Apple M1 GPU

In [5]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [6]:
# Define model
class NeuralNetwork(nn.Module): # nn.Module 클래스를 상속 # NeuralNetwork이 자식클래스
    def __init__(self): # 뉴럴 네트워크의 성분을 정의
        super().__init__() # 부모 클래스의 초기화 함수를 호출해서 속성을 초기화, 필요한 기능 추가
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


---

### Optimizing the Model Parameters
To train a model, we need a loss function and an optimizer.

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches),   
and backpropagates the prediction error to adjust the model’s parameters.  
단일 훈련 루프에서 모델은 훈련 데이터세트(일괄적으로 입력)에 대해 예측을 수행하고 예측 오류를 역전파하여 모델의 매개변수를 조정

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y) # loss_fn = nn.CrossEntropyLoss() 지정했음

        # Backpropagation
        loss.backward() # 역전파
        optimizer.step() # 최적화 함수를 사용하여 모델의 매개변수 업데이트
        optimizer.zero_grad() 

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

We also check the model’s performance against the test dataset to ensure it is learning.  
또한 테스트 데이터 세트와 비교하여 모델 성능을 확인하여 학습 중인지 확인

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (epochs).   
훈련 과정은 여러 반복(epoch)에 걸쳐 수행  

During each epoch, the model learns parameters to make better predictions.   
각 epoch마다 모델은 더 나은 예측을 위해 매개변수를 학습  
에포크를 세대라고 번역하기도 함 (책에서도 세대라는 용어 나왔음)

We print the model’s accuracy and loss at each epoch;   
각 epoch마다 모델의 정확도와 손실을 인쇄  

we’d like to see the accuracy increase and the loss decrease with every epoch.  
매 에포크마다 정확도가 증가하고 손실이 감소하는 것을 예상

In [10]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302471  [   64/60000]
loss: 2.287063  [ 6464/60000]
loss: 2.271568  [12864/60000]
loss: 2.265818  [19264/60000]
loss: 2.237872  [25664/60000]
loss: 2.219079  [32064/60000]
loss: 2.229588  [38464/60000]
loss: 2.188260  [44864/60000]
loss: 2.185202  [51264/60000]
loss: 2.164438  [57664/60000]
Test Error: 
 Accuracy: 49.1%, Avg loss: 2.146684 

Epoch 2
-------------------------------
loss: 2.156263  [   64/60000]
loss: 2.142411  [ 6464/60000]
loss: 2.081804  [12864/60000]
loss: 2.099488  [19264/60000]
loss: 2.039707  [25664/60000]
loss: 1.988637  [32064/60000]
loss: 2.025023  [38464/60000]
loss: 1.930287  [44864/60000]
loss: 1.942073  [51264/60000]
loss: 1.880179  [57664/60000]
Test Error: 
 Accuracy: 56.1%, Avg loss: 1.863148 

Epoch 3
-------------------------------
loss: 1.899779  [   64/60000]
loss: 1.867910  [ 6464/60000]
loss: 1.740246  [12864/60000]
loss: 1.785234  [19264/60000]
loss: 1.672924  [25664/60000]
loss: 1.635085  [32064/600

---

### Saving Models
A common way to save a model is to serialize the internal state dictionary (containing the model parameters).  
모델을 저장하는 일반적인 방법은 내부 상태 사전(모델 매개변수 포함)을 직렬화하는 것  
internal state dictionary 내부 상태 사전으로 번역됨

In [11]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


### Loading Models
The process for loading a model includes re-creating the model structure and loading the state dictionary into it.  
모델을 로드하는 프로세스에는 모델 구조를 다시 만들고 상태 사전을 해당 구조에 로드하는 작업이 포함

In [12]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth")) # 이후 모델을 업데이트하려면 모델에 대한 정보가 있어야 함

<All keys matched successfully>

This model can now be used to make predictions.  
이제 이 모델을 사용하여 예측

In [13]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
